In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('bigcode/bigcodebench')

dataset


DatasetDict({
    v0.1.0_hf: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
    v0.1.1: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
})

In [7]:
dataset=dataset.shuffle(seed=42)

In [8]:
dataset

DatasetDict({
    v0.1.0_hf: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
    v0.1.1: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
})

In [9]:
def transform_to_llama(example):
    instruct_prompt = example['instruct_prompt'].strip()
    complete_prompt = example['complete_prompt'].strip()

    # Apply the LLaMA 2 template
    transformed_prompt = f'<s>[INST] {instruct_prompt} [/INST] {complete_prompt} </s>'

    return {'text': transformed_prompt}


In [10]:
transformed_dataset = dataset.map(transform_to_llama)

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [ ]:
# Shuffle the dataset and select a subset for faster processing
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

# Define a function to transform the data into the LLaMA 2 fine-tuning format
def transform_to_llama(example):
    instruct_prompt = example['instruct_prompt'].strip()
    complete_prompt = example['complete_prompt'].strip()

    # Apply the LLaMA 2 template
    transformed_prompt = f'<s>[INST] {instruct_prompt} [/INST] {complete_prompt} </s>'

    return {'text': transformed_prompt}

# Apply the transformation
transformed_dataset = dataset.map(transform_to_llama)

In [11]:
transformed_dataset

DatasetDict({
    v0.1.0_hf: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs', 'text'],
        num_rows: 1140
    })
    v0.1.1: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs', 'text'],
        num_rows: 1140
    })
})

In [18]:

from huggingface_hub import notebook_login

notebook_login()




In [19]:
transformed_dataset.push_to_hub("saishshinde15/saish-1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/saishshinde15/saish-1k/commit/afdc0b1669b1b9b15ab63c4ecf2a6ebf1359521c', commit_message='Upload dataset', commit_description='', oid='afdc0b1669b1b9b15ab63c4ecf2a6ebf1359521c', pr_url=None, pr_revision=None, pr_num=None)